In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import wrds

# -------------------
# USER CONFIG
# -------------------
WRDS_USERNAME = "acaraman"
LIB = "optionm"

TICKER = "AAPL"
START_DATE = "2016-01-01"
END_DATE   = "2025-12-31"

YEARS = list(range(int(START_DATE[:4]), int(END_DATE[:4]) + 1))

# -------------------
# OUTPUT PATHS
# -------------------
RAW_DIR = Path("../../Data/raw")
RAW_DIR.mkdir(parents=True, exist_ok=True)

print("Saving to:", RAW_DIR.resolve())

Saving to: C:\Users\Admin\OneDrive\Desktop\Fifth Year\Computer Science\CS4490 Thesis\Codebase\Thesis\Data\raw


In [2]:
db = wrds.Connection(wrds_username=WRDS_USERNAME)

Loading library list...
Done


In [3]:
def list_tables(lib=LIB):
    return set(db.list_tables(library=lib))

def describe_cols(table, lib=LIB):
    desc = db.describe_table(library=lib, table=table)
    return desc["name"].tolist(), desc

def pick_col(cols, candidates, required=True):
    cols_l = [c.lower() for c in cols]
    for cand in candidates:
        cand_l = cand.lower()
        if cand_l in cols_l:
            return cols[cols_l.index(cand_l)]
    if required:
        raise KeyError(f"Could not find any of {candidates} in columns: {cols}")
    return None

def has_table(table, lib=LIB):
    return table in list_tables(lib)


In [4]:
def resolve_secids_for_ticker(ticker: str, start_date: str, end_date: str, lib=LIB) -> pd.DataFrame:
    """
    Returns a DataFrame with a column 'secid' containing candidate secids for the ticker.
    Tries:
      1) optionm.secnmd with a direct ticker column (ticker/tic/symbol)
      2) optionm.secnmd joined to optionm.ticker (tickerid -> ticker text)
    """
    ticker = ticker.upper()

    if not has_table("secnmd", lib):
        raise RuntimeError(f"{lib}.secnmd not found. Available tables may differ.")

    secnmd_cols, _ = describe_cols("secnmd", lib)

    secid_col = pick_col(secnmd_cols, ["secid"], required=True)
    # try common ticker string columns
    tkr_col = pick_col(secnmd_cols, ["ticker", "tic", "symbol"], required=False)

    # optional effective-date style columns
    eff_col = pick_col(secnmd_cols, ["date", "dt", "effectivedate", "effective_date", "effdate", "eff_date"], required=False)
    end_col = pick_col(secnmd_cols, ["enddate", "end_date", "thrudate", "thru_date"], required=False)

    # Build an overlap filter if effective date ranges exist
    # We want records that overlap [start_date, end_date]
    overlap_filter = ""
    if eff_col and end_col:
        overlap_filter = f"""
          AND {eff_col} <= '{end_date}'
          AND ({end_col} IS NULL OR {end_col} >= '{start_date}')
        """
    elif eff_col:
        overlap_filter = f" AND {eff_col} <= '{end_date}'"

    # Case 1: direct ticker string column exists
    if tkr_col:
        q = f"""
        SELECT DISTINCT {secid_col} AS secid
        FROM {lib}.secnmd
        WHERE {tkr_col} = '{ticker}'
        {overlap_filter}
        """
        out = db.raw_sql(q)
        if not out.empty:
            return out

    # Case 2: join through ticker table if secnmd has tickerid
    tickerid_col = pick_col(secnmd_cols, ["tickerid", "ticker_id"], required=False)
    if tickerid_col and has_table("ticker", lib):
        t_cols, _ = describe_cols("ticker", lib)
        t_id = pick_col(t_cols, ["tickerid", "ticker_id"], required=True)
        t_txt = pick_col(t_cols, ["ticker", "tic", "symbol"], required=True)

        q = f"""
        SELECT DISTINCT sn.{secid_col} AS secid
        FROM {lib}.secnmd sn
        JOIN {lib}.ticker t
          ON sn.{tickerid_col} = t.{t_id}
        WHERE t.{t_txt} = '{ticker}'
        {overlap_filter}
        """
        out = db.raw_sql(q)
        if not out.empty:
            return out

    raise RuntimeError(
        "Could not resolve secid for ticker. "
        "Inspect optionm.secnmd and optionm.ticker schema to adjust mapping."
    )

secids = resolve_secids_for_ticker(TICKER, START_DATE, END_DATE)
secids


Approximately 271874 rows in optionm.secnmd.


,secid
0,101594.0


In [5]:
def choose_best_secid_by_surface_coverage(secids_df: pd.DataFrame, years=YEARS, lib=LIB) -> int:
    """
    If multiple secids exist for a ticker, choose the one with the most distinct surface dates
    across vsurfdYYYY tables within [START_DATE, END_DATE].
    """
    if secids_df.empty:
        raise RuntimeError("No secids provided.")

    secid_list = ",".join(str(int(x)) for x in secids_df["secid"].unique())

    counts = []
    for y in years:
        t = f"vsurfd{y}"
        if not has_table(t, lib):
            continue
        q = f"""
        SELECT secid, COUNT(DISTINCT date) AS n_days
        FROM {lib}.{t}
        WHERE secid IN ({secid_list})
          AND date BETWEEN '{START_DATE}' AND '{END_DATE}'
        GROUP BY secid
        """
        counts.append(db.raw_sql(q))

    if not counts:
        raise RuntimeError("No vsurfdYYYY tables found in the selected years.")

    cov = (pd.concat(counts, ignore_index=True)
             .groupby("secid", as_index=False)["n_days"].sum()
             .sort_values("n_days", ascending=False))

    display(cov.head(10))
    return int(cov.iloc[0]["secid"])

SECID = choose_best_secid_by_surface_coverage(secids)
print("Chosen SECID:", SECID)


,secid,n_days
0,101594.0,2429


Chosen SECID: 101594


In [6]:
def pull_surface_for_secid(secid: int, years=YEARS, lib=LIB) -> pd.DataFrame:
    frames = []
    for y in years:
        t = f"vsurfd{y}"
        if not has_table(t, lib):
            continue

        q = f"""
        SELECT
            secid,
            date,
            days,
            delta,
            cp_flag,
            impl_volatility,
            impl_strike,
            impl_premium,
            dispersion
        FROM {lib}.{t}
        WHERE secid = {secid}
          AND date BETWEEN '{START_DATE}' AND '{END_DATE}'
        """
        frames.append(db.raw_sql(q, date_cols=["date"]))

    if not frames:
        raise RuntimeError("No surface data pulled. Check SECID/date range.")

    vs = pd.concat(frames, ignore_index=True)
    return vs

vs = pull_surface_for_secid(SECID)
print("Surface shape:", vs.shape)
vs.head()


Surface shape: (908446, 9)


,secid,date,days,delta,cp_flag,impl_volatility,impl_strike,impl_premium,dispersion
0,101594.0,2016-01-04,10.0,-90.0,P,0.351577,113.5996,8.665669,0.214947
1,101594.0,2016-01-04,10.0,-85.0,P,0.296881,110.8722,6.048973,0.144321
2,101594.0,2016-01-04,10.0,-80.0,P,0.267112,109.3283,4.620924,0.069316
3,101594.0,2016-01-04,10.0,-75.0,P,0.261231,108.4435,3.891608,0.023896
4,101594.0,2016-01-04,10.0,-70.0,P,0.263302,107.7605,3.397845,0.007847


In [7]:
def pull_underlying_prices(secid: int, years=YEARS, lib=LIB) -> pd.DataFrame:
    """
    Tries multiple common IvyDB/WRDS representations:
      1) optionm.security_price if it contains secid
      2) optionm.secprd (often a union/view) if it contains secid
      3) optionm.secprdYYYY tables (yearly)
    Returns a DataFrame with at least ['secid','date', ...]
    """
    # 1) security_price
    if has_table("security_price", lib):
        cols, _ = describe_cols("security_price", lib)
        id_col = pick_col(cols, ["secid", "securityid"], required=False)
        date_col = pick_col(cols, ["date", "dt"], required=False)

        if id_col and date_col:
            # If it's keyed by secid, we're done
            if id_col.lower() == "secid":
                q = f"""
                SELECT *
                FROM {lib}.security_price
                WHERE {id_col} = {secid}
                  AND {date_col} BETWEEN '{START_DATE}' AND '{END_DATE}'
                """
                return db.raw_sql(q, date_cols=["date"])

            # If it's keyed by securityid, we try to map secid -> securityid using optionm.security
            if id_col.lower() == "securityid" and has_table("security", lib):
                s_cols, _ = describe_cols("security", lib)
                s_secid = pick_col(s_cols, ["secid"], required=False)
                s_securityid = pick_col(s_cols, ["securityid"], required=False)

                if s_secid and s_securityid:
                    q_map = f"""
                    SELECT DISTINCT {s_securityid} AS securityid
                    FROM {lib}.security
                    WHERE {s_secid} = {secid}
                    """
                    m = db.raw_sql(q_map)
                    if not m.empty:
                        securityid = int(m.iloc[0]["securityid"])
                        q = f"""
                        SELECT *
                        FROM {lib}.security_price
                        WHERE {id_col} = {securityid}
                          AND {date_col} BETWEEN '{START_DATE}' AND '{END_DATE}'
                        """
                        out = db.raw_sql(q, date_cols=["date"])
                        out["secid"] = secid
                        return out

    # 2) secprd (often a convenient union/view)
    if has_table("secprd", lib):
        cols, _ = describe_cols("secprd", lib)
        id_col = pick_col(cols, ["secid"], required=False)
        date_col = pick_col(cols, ["date", "dt"], required=False)
        if id_col and date_col:
            q = f"""
            SELECT *
            FROM {lib}.secprd
            WHERE {id_col} = {secid}
              AND {date_col} BETWEEN '{START_DATE}' AND '{END_DATE}'
            """
            return db.raw_sql(q, date_cols=["date"])

    # 3) yearly secprdYYYY tables
    frames = []
    for y in years:
        t = f"secprd{y}"
        if not has_table(t, lib):
            continue
        cols, _ = describe_cols(t, lib)
        id_col = pick_col(cols, ["secid"], required=False)
        date_col = pick_col(cols, ["date", "dt"], required=False)
        if not (id_col and date_col):
            continue
        q = f"""
        SELECT *
        FROM {lib}.{t}
        WHERE {id_col} = {secid}
          AND {date_col} BETWEEN '{START_DATE}' AND '{END_DATE}'
        """
        frames.append(db.raw_sql(q, date_cols=["date"]))

    if frames:
        return pd.concat(frames, ignore_index=True)

    raise RuntimeError("Could not pull underlying prices from security_price/secprd/secprdYYYY tables.")

px = pull_underlying_prices(SECID)
print("Price shape:", px.shape)
px.head()


Approximately 132438384 rows in optionm.security_price.
Approximately 96285 rows in optionm.security.
Approximately 65838324 rows in optionm.secprd.
Price shape: (2429, 11)


,secid,date,low,high,close,volume,return,cfadj,open,cfret,shrout
0,101594.0,2016-01-04,102.0,105.368,105.35,67634870.0,0.000855,28.0,102.61,30.07141,5575331.0
1,101594.0,2016-01-05,102.41,105.85,102.71,55769224.0,-0.025059,28.0,105.75,30.07141,5575331.0
2,101594.0,2016-01-06,99.87,102.37,100.7,68441828.0,-0.01957,28.0,100.56,30.07141,5575331.0
3,101594.0,2016-01-07,96.43,100.13,96.45,81036368.0,-0.042205,28.0,98.68,30.07141,5575331.0
4,101594.0,2016-01-08,96.76,99.11,96.96,70786622.0,0.005288,28.0,98.55,30.07141,5575331.0


In [9]:
# Normalize delta: if it's 10..90, convert to 0.10..0.90
if vs["delta"].max() > 2:
    vs["delta"] = vs["delta"] / 100.0

# Cast days to int when safe
vs["days"] = vs["days"].astype(int)

# Ensure cp_flag uppercase
vs["cp_flag"] = vs["cp_flag"].str.upper()

vs[["date","days","delta","cp_flag","impl_volatility","dispersion"]].head()


,date,days,delta,cp_flag,impl_volatility,dispersion
0,2016-01-04,10,-0.9,P,0.351577,0.214947
1,2016-01-04,10,-0.85,P,0.296881,0.144321
2,2016-01-04,10,-0.8,P,0.267112,0.069316
3,2016-01-04,10,-0.75,P,0.261231,0.023896
4,2016-01-04,10,-0.7,P,0.263302,0.007847


In [10]:
print("Surface date range:", vs["date"].min(), "->", vs["date"].max())
print("Unique dates:", vs["date"].nunique())
print("Unique days:", vs["days"].nunique())
print("Unique deltas:", vs["delta"].nunique())
print("cp_flag:", sorted(vs["cp_flag"].unique()))

print("\nDispersion == -99.99 proportion:", (vs["dispersion"] == -99.99).mean())

rows_per_day = vs.groupby("date").size()
print("\nRows per date (describe):")
display(rows_per_day.describe())

# Check grid stability (days x delta x cp)
unique_days = vs["days"].nunique()
unique_deltas = vs["delta"].nunique()
expected_per_day = unique_days * unique_deltas * vs["cp_flag"].nunique()
print("\nIf full grid were present each day, expected rows/day =", expected_per_day)

# A quick missingness view: how many (days,delta,cp) points per day?
grid_points_per_day = vs.groupby(["date"]).apply(lambda g: g[["days","delta","cp_flag"]].drop_duplicates().shape[0])
print("\nDistinct grid points per day (describe):")
display(grid_points_per_day.describe())


Surface date range: 2016-01-04 00:00:00 -> 2025-08-29 00:00:00
Unique dates: 2429
Unique days: 11
Unique deltas: 34
cp_flag: ['C', 'P']

Dispersion == -99.99 proportion: 0.0

Rows per date (describe):


count    2429.0
mean      374.0
std         0.0
min       374.0
25%       374.0
50%       374.0
75%       374.0
max       374.0
dtype: float64


If full grid were present each day, expected rows/day = 748

Distinct grid points per day (describe):


C:\Users\Admin\AppData\Local\Temp\ipykernel_17340\810184358.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grid_points_per_day = vs.groupby(["date"]).apply(lambda g: g[["days","delta","cp_flag"]].drop_duplicates().shape[0])


count    2429.0
mean      374.0
std         0.0
min       374.0
25%       374.0
50%       374.0
75%       374.0
max       374.0
dtype: float64

In [11]:
vs_out = RAW_DIR / f"{TICKER}_vsurfd_{START_DATE}_{END_DATE}.csv.gz"
px_out = RAW_DIR / f"{TICKER}_underlying_{START_DATE}_{END_DATE}.csv.gz"

vs.to_csv(vs_out, index=False, compression="gzip")
px.to_csv(px_out, index=False, compression="gzip")

print("Saved surface ->", vs_out)
print("Saved underlying ->", px_out)


Saved surface -> ..\..\Data\raw\AAPL_vsurfd_2016-01-01_2025-12-31.csv.gz
Saved underlying -> ..\..\Data\raw\AAPL_underlying_2016-01-01_2025-12-31.csv.gz


In [12]:
db.close()